# Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
import pandas as pd

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

# Scraping

## NASA Mars News

In [3]:
# Retrieve the latest news title
news_title = soup.find_all('div', class_='content_title')[0].text

# Retrieve the latest news paragraph
news_paragraph = soup.find_all('div', class_='rollover_description_inner')[0].text

print(f"The latest News Title and Paragraph Text from Nasa's Mars Exploration website is {news_title} {news_paragraph}")

The latest News Title and Paragraph Text from Nasa's Mars Exploration website is 

NASA's Mars Perseverance Rover Provides Front-Row Seat to Landing, First Audio Recording of Red Planet 

 
The agency’s newest rover captured first-of-its kind footage of its Feb. 18 touchdown and has recorded audio of Martian wind.





## JPL Mars Space Images - Featured Image

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/lhabersham/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [5]:
jpl_url = 'https://www.jpl.nasa.gov/images?search=&category=Mars'
browser.visit(jpl_url)

In [6]:
for x in range(1, 2):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    imgs = soup.find_all('div', class_='SearchResultCard')[0]

# Use Beautiful Soup's find() method to navigate and retrieve attributes
    for img in imgs:
        link = imgs.find('a')
        href = link['href']
        print('-----------')
        print(img.text)
        print('https://www.jpl.nasa.gov' + href)

-----------

        Image
        .   
          Image
          .   Kaiser Crater Dunes  Feb 26, 2021  
https://www.jpl.nasa.gov/images/kaiser-crater-dunes-8


In [7]:
main_url = 'https://www.jpl.nasa.gov'
featured_image_url = main_url + href
print(featured_image_url)

https://www.jpl.nasa.gov/images/kaiser-crater-dunes-8


## Mars Facts

In [8]:
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)

In [9]:
mars_fact = pd.read_html(mars_facts_url)
mars_fact

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [10]:
#Find Mars Facts DataFrame in the lists of DataFrames
mars_facts = mars_fact[0]
# mars_facts

#Assign the columns
mars_facts.columns = ['Description', 'Value']
# mars_facts

mars_facts

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [11]:
mars_facts.to_html('mars_facts.html', index=False, classes=['table', 'table-striped', 'table-hover'])

In [12]:
print(mars_facts)

            Description                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


## Mars Hemispheres

In [13]:
# Scrape Mars hemispheres titles and images
mh_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
mh_img_url = 'https://astrogeology.usgs.gov'

browser.visit(mh_url)

In [14]:
# Extract hemispheres item elements. This will help with scraping the correct items you need.
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_hemis = soup.find('div',class_='collapsible results')
mars_items = mars_hemis.find_all('div',class_='item')

In [15]:
#EXTRACTION
mars_hemi_img_url = []

for items in mars_items:
    # Extract title
    hemi = items.find('div',class_='description')
    title = hemi.h3.text
    
    # Extract image url
    hemi_url = hemi.a['href']
    browser.visit(mh_img_url + hemi_url)
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    image_src = soup.find('li').a['href']
    
    if (title and image_src):
        # Print results
        print('-'*50)
        print(title)
        print(image_src)
    # Create dictionary for title and url
    hemi_dict={
        'title':title,
        'image_url':image_src
    }
    mars_hemi_img_url.append(hemi_dict)

--------------------------------------------------
Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
--------------------------------------------------
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
--------------------------------------------------
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
--------------------------------------------------
Valles Marineris Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [16]:
# Create dictionary for all info scraped from sources above to support your data in Mongo
mars_facts_dict = {
    "news_title":news_title,
    "news_paragraph":news_paragraph,
    "featured_image_url":featured_image_url,
    "mars_fact_table":mars_facts,
    "hemisphere_images":mars_hemi_img_url
}

mars_facts_dict



{'news_title': "\n\nNASA's Mars Perseverance Rover Provides Front-Row Seat to Landing, First Audio Recording of Red Planet \n\n",
 'news_paragraph': '\nThe agency’s newest rover captured first-of-its kind footage of its Feb. 18 touchdown and has recorded audio of Martian wind.\n\n\n',
 'featured_image_url': 'https://www.jpl.nasa.gov/images/kaiser-crater-dunes-8',
 'mars_fact_table':             Description                          Value
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
 'hemisphere_images': [{'title': 'Cerberus Hemisphere 

In [17]:
browser.quit()